In [ ]:
import tensorflow as tf
import tensorflow_datasets as tfds
import numpy as np
import pickle
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.regularizers import l2
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping, ModelCheckpoint

# Load EMNIST ByClass dataset
(ds_train, ds_test), ds_info = tfds.load(
    'emnist/byclass',
    split=['train', 'test'],
    shuffle_files=True,
    as_supervised=True,
    with_info=True,
)

# Preprocessing function
def preprocess(image, label):
    image = tf.cast(image, tf.float32) / 255.0
    image = tf.image.transpose(image)  # Required for EMNIST
    return image, label

# Prepare datasets
batch_size = 128
ds_train = ds_train.map(preprocess).cache().shuffle(10000).batch(batch_size).prefetch(tf.data.AUTOTUNE)
ds_test = ds_test.map(preprocess).batch(batch_size).prefetch(tf.data.AUTOTUNE)

# Build enhanced CNN model
model = Sequential([
    Conv2D(64, (3, 3), activation='relu', padding='same',
           input_shape=(28, 28, 1), kernel_regularizer=l2(1e-4)),
    BatchNormalization(),
    MaxPooling2D(pool_size=(2, 2)),
    Dropout(0.25),

    Conv2D(128, (3, 3), activation='relu', padding='same', kernel_regularizer=l2(1e-4)),
    BatchNormalization(),
    MaxPooling2D(pool_size=(2, 2)),
    Dropout(0.3),

    Conv2D(256, (3, 3), activation='relu', padding='same', kernel_regularizer=l2(1e-4)),
    BatchNormalization(),
    MaxPooling2D(pool_size=(2, 2)),
    Dropout(0.4),

    Flatten(),
    Dense(512, activation='relu', kernel_regularizer=l2(1e-4)),
    Dropout(0.5),
    Dense(ds_info.features['label'].num_classes, activation='softmax')
])

# Compile model
model.compile(optimizer=Adam(learning_rate=0.0005),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# Callbacks
callbacks = [
    ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=2, min_lr=1e-6),
    EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True),
    ModelCheckpoint("EMNIST_V2_best_model_best.h5", save_best_only=True)
]

# Print model summary
model.summary()

# Train model
history = model.fit(ds_train,
                    validation_data=ds_test,
                    epochs=30,
                    callbacks=callbacks)

# Save final model and history
model.save('EMNIST_V2_model.h5')
print("✅ Model saved as EMNIST_V2_model.h5")

with open('EMNIST_V2_history.pkl', 'wb') as f:
    pickle.dump(history.history, f)
print("📊 Training history saved as emnist_byclass_history_v2.pkl")


c:\Users\Hashtag\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\layers\convolutional\base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 28, 28, 64)     │           640 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 28, 28, 64)     │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 14, 14, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 14, 14, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 14, 14, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 14, 14, 128)    │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 7, 7, 128)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 7, 7, 128)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 7, 7, 256)      │       295,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_2           │ (None, 7, 7, 256)      │         1,024 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 3, 3, 256)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 3, 3, 256)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 2304)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 512)            │     1,180,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_3 (Dropout)             │ (None, 512)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 62)             │        31,806 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,583,422 (6.04 MB)

 Trainable params: 1,582,526 (6.04 MB)

...

5453/5453 ━━━━━━━━━━━━━━━━━━━━ 1286s 236ms/step - accuracy: 0.8705 - loss: 0.4061 - val_accuracy: 0.8763 - val_loss: 0.3774 - learning_rate: 1.2500e-04
Epoch 26/30
5453/5453 ━━━━━━━━━━━━━━━━━━━━ 0s 228ms/step - accuracy: 0.8704 - loss: 0.3999

5453/5453 ━━━━━━━━━━━━━━━━━━━━ 1278s 234ms/step - accuracy: 0.8704 - loss: 0.3999 - val_accuracy: 0.8782 - val_loss: 0.3715 - learning_rate: 1.2500e-04
Epoch 27/30
5453/5453 ━━━━━━━━━━━━━━━━━━━━ 0s 227ms/step - accuracy: 0.8708 - loss: 0.3960

5453/5453 ━━━━━━━━━━━━━━━━━━━━ 1275s 234ms/step - accuracy: 0.8708 - loss: 0.3960 - val_accuracy: 0.8774 - val_loss: 0.3696 - learning_rate: 1.2500e-04
Epoch 28/30
5453/5453 ━━━━━━━━━━━━━━━━━━━━ 0s 228ms/step - accuracy: 0.8711 - loss: 0.3929

5453/5453 ━━━━━━━━━━━━━━━━━━━━ 1280s 235ms/step - accuracy: 0.8711 - loss: 0.3929 - val_accuracy: 0.8786 - val_loss: 0.3666 - learning_rate: 1.2500e-04
Epoch 29/30
5453/5453 ━━━━━━━━━━━━━━━━━━━━ 1281s 235ms/step - accuracy: 0.8713 - loss: 0.3912 - val_accuracy: 0.8781 - val_loss: 0.3679 - learning_rate: 1.2500e-04
Epoch 30/30
5453/5453 ━━━━━━━━━━━━━━━━━━━━ 1287s 236ms/step - accuracy: 0.8713 - loss: 0.3904 - val_accuracy: 0.8784 - val_loss: 0.3677 - learning_rate: 1.2500e-04


✅ Model saved as emnist_byclass_cnn_model_v2.h5
📊 Training history saved as emnist_byclass_history_v2.pkl
